In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Part 1: Load and Create Dataframe from Wiki Page

### Create a list from the data on wiki page

In [2]:
import pandas as pd    
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'    
toronto_wiki = pd.read_html(page, index_col=0, attrs={"class":"wikitable"})     

### Place list's datas into a dataframe

In [3]:
toronto_wiki_df = pd.DataFrame()
toronto_wiki_df

for data in toronto_wiki:
     toronto_wiki_df= toronto_wiki_df.append(data)
toronto_wiki_df=toronto_wiki_df.reset_index()
toronto_wiki_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Rename "Postcode" to "PostalCode" and "Neighbourhood" to "Neighborhood"

In [4]:
toronto_wiki_df=toronto_wiki_df.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"})
toronto_wiki_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Turn any non-assigned borough into NaN then drop those data point that have NaN borough

In [5]:
toronto_wiki_df.loc[toronto_wiki_df['Borough'] == 'Not assigned','Borough'] = np.nan
toronto_wiki_df.dropna(inplace=True)

### Simplify the dataframe

In [6]:
toronto_neighborhoods=toronto_wiki_df.groupby('PostalCode').agg({'Borough':'first','Neighborhood': ', '.join})

### Reset index

In [7]:
toronto_neighborhoods=toronto_neighborhoods.reset_index() 
toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
toronto_neighborhoods.shape

(103, 3)

## Part 2: Geographical Coordinates 

### Create dataframe contain Lat and Long for each postal code

In [9]:
geo_data=pd.read_csv("http://cocl.us/Geospatial_data")
geo_data=geo_data.rename(columns={"Postal Code": "PostalCode"})
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Create a empty lat dataframe then add it to the result-dataframe from part 1

In [10]:
lat=pd.DataFrame(columns=["Latitude","Longitude"])
toronto_neighborhoods=pd.concat([toronto_neighborhoods, lat], axis=1, sort=False)
toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


### Find the lat and long from geo dataframe and add to the corresponding postal code in the main database

In [11]:
for i in range(toronto_neighborhoods.shape[0]):
    pc=toronto_neighborhoods.PostalCode[i]
    for j in range(geo_data.shape[0]):
        if geo_data.PostalCode[j]==pc:
            toronto_neighborhoods.Latitude[i]=geo_data.Latitude[j]
            toronto_neighborhoods.Longitude[i]=geo_data.Longitude[j]
            break
toronto_neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


## Part 3:Toronto's Neighborhood Exploration and Cluster